In [ ]:
 byimport os                                                                       #OS for path of folders
import cv2                                                                      #cv2 for reading the images
import numpy as np                                                              #numpy for numpy arrays
import tensorflow as tf                                                         #tensorflow for the convolutional neural network model
from sklearn.model_selection import train_test_split                            #spliting the training set
from tensorflow.keras import layers, models, callbacks                          #adding custom layers to our model
from sklearn.preprocessing import LabelEncoder                                  #assigning proper labels to our predictions

from tensorflow.keras.applications import MobileNetV2                           #our model is based on MobileNetV2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir_AI = "/content/drive/MyDrive/ai"                     #data directories for ai and real images
data_dir_Real = "/content/drive/MyDrive/real"

images = []                                                   #lists where images and labels will be stored
labels = []

for data_dir in [data_dir_AI, data_dir_Real]:
    file_paths = [os.path.join(data_dir, file) for file in os.listdir(data_dir)]
    for file_path in file_paths:
        if file_path.endswith('.jpg') or file_path.endswith('.png'):
            image = cv2.imread(file_path)
            if image is not None:
                image = cv2.resize(image, (128, 128))                                   #resizing all the images to the same size of 128x128
                images.append(image)                                              #appending images a nd their respective labels , where real is 1 and ai is 0
                labels.append(0 if data_dir == data_dir_AI else 1)

# Convert lists to numpy arrays
images = np.array(images)                                                         #converting list to numpy arrays
labels = np.array(labels)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)  # split the dataset into training and testing sets

train_images = train_images / 255.0                                                                                      #resccaling the pixel values
test_images = test_images / 255.0

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

for layer in base_model.layers:
    layer.trainable = False


model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])


opt = tf.keras.optimizers.Adam(learning_rate=0.0001)                    #compiling the model
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, epochs=25, validation_data=(test_images, test_labels))          #training the model

Epoch 1/25
32/32 [==============================] - 21s 578ms/step - loss: 0.7230 - accuracy: 0.5488 - val_loss: 0.6489 - val_accuracy: 0.6295
Epoch 2/25
32/32 [==============================] - 16s 506ms/step - loss: 0.5964 - accuracy: 0.6882 - val_loss: 0.6146 - val_accuracy: 0.7012
Epoch 3/25
32/32 [==============================] - 14s 449ms/step - loss: 0.5504 - accuracy: 0.7201 - val_loss: 0.5882 - val_accuracy: 0.7131
Epoch 4/25
32/32 [==============================] - 14s 452ms/step - loss: 0.4892 - accuracy: 0.7749 - val_loss: 0.5904 - val_accuracy: 0.7211
Epoch 5/25
32/32 [==============================] - 14s 447ms/step - loss: 0.4450 - accuracy: 0.8018 - val_loss: 0.6028 - val_accuracy: 0.7131
Epoch 6/25
32/32 [==============================] - 14s 447ms/step - loss: 0.4305 - accuracy: 0.8058 - val_loss: 0.5689 - val_accuracy: 0.7251
Epoch 7/25
32/32 [==============================] - 14s 436ms/step - loss: 0.4035 - accuracy: 0.8396 - val_loss: 0.6031 - val_accuracy: 0.7052

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)                #sample test with the model
print('Test accuracy:', test_acc)


8/8 [==============================] - 2s 293ms/step - loss: 0.5884 - accuracy: 0.7371
Test accuracy: 0.737051784992218


In [ ]:
import pandas as pd

def load_dataset(dataset_dir):
    images = []
    filenames = []
    for filename in os.listdir(dataset_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = cv2.imread(os.path.join(dataset_dir, filename))
            img = cv2.resize(img, (128, 128))                    #resizing the test images to same as the ones in training set
            img = img / 255.0                                    #rescaling pixel intensities
            images.append(img)
            filenames.append(filename)
    return np.array(images), filenames

dataset_dir = "/content/drive/MyDrive/Test_Images"              #loading the dataset
X, filenames = load_dataset(dataset_dir)
predictions = model.predict(X)                                  #predicting values using model

In [ ]:
results_df = pd.DataFrame({'Prediction': predictions[:, 0]})                    # create a DataFrame to store predictions and filenames

results_df['Id'] = [int(x.split('_')[1].split('.')[0]) for x in filenames]                  # Sort the DataFrame by filenames
results_df = results_df.sort_values(by='Id').reset_index(drop=True)

results_df['Id'] = results_df['Id'].apply(lambda x: f'image_{x}')                 #removing 'jpg'

results_df['Label'] = results_df['Prediction'].apply(lambda x: 'Real' if x >= 0.5 else 'AI')        #assigning predicitons to labels

results_df.drop(['Prediction'], axis=1, inplace=True)                                   #removing columns which are not needed

results_df.to_csv("/content/drive/MyDrive/predictions12345.csv", header=False, index=False)

output_csv_path = "/content/drive/MyDrive/predictions12345.csv"            # output results to a CSV file
results_df.to_csv(output_csv_path, index=False)

print("Predictions saved to", output_csv_path)
csv_file_path = "/content/drive/MyDrive/predictions12345.csv"

predictions_df = pd.read_csv(csv_file_path)                       #csv file read by pandas dataframe

print(predictions_df.head())  # Display the first few rows

16/16 [==============================] - 5s 256ms/step
            Id Label
0      image_1  Real
1      image_2    AI
2      image_3    AI
3      image_4  Real
4      image_5    AI
..         ...   ...
495  image_496  Real
496  image_497  Real
497  image_498  Real
498  image_499  Real
499  image_500  Real

[500 rows x 2 columns]
Predictions saved to /content/drive/MyDrive/predictions12345.csv
        Id Label
0  image_1  Real
1  image_2    AI
2  image_3    AI
3  image_4  Real
4  image_5    AI
             Id Label
count       500   500
unique      500     2
top     image_1  Real
freq          1   279
